# [SemEval 2023 Task 1](https://raganato.github.io/vwsd/)

[By Abdullah Alshadadi (Srking501)](https://github.com/Srking501)

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import keras
import random
import os

2023-03-17 00:01:39.897118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 00:01:40.440663: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-17 00:01:40.440706: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-17 00:01:40.440711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# to access google drive folder
# from google.colab import drive
# drive.mount('/content/drive') # When you run this you'll be prompted for a token - follow the link to generate this.
# #        "/content/drive/MyDrive/...path/to/dir
# path = "/content/drive/MyDrive/semeval-2023/task-1/"
# path = "/content/drive/MyDrive/data/"

# Comment this out if you are using Google Colab, otherwise just change to local directory where the data is located
path = "./data/"

In [4]:
os.getcwd()

'/home/srking501/Desktop/Projects/semeval-2023-task1'

❗❗❗Retrieve the data from [SemEval-2023 Task-1 page](https://raganato.github.io/vwsd/), specifically the ["[TRAIN+TRIAL]" data](https://drive.google.com/file/d/1byX4wpe1UjyCVyYrT04sW17NnycKAK7N/view), and put it into `data/` directory❗❗❗

In [5]:
from zipfile import ZipFile

try:
    if os.path.isdir(path + "semeval-2023-task-1-V-WSD-train-v1") == False:
        with ZipFile(path + "semeval-2023-task-1-V-WSD-train-v1.zip", "r") as zip:
            print("Unzipping data...")
            zip.extractall("data/")
            print("Finished.")
    else:
        print("[Data already unzipped]")
except FileNotFoundError:
    print(f"The file semeval-2023-task-1-V-WSD-train-v1.zip is not found in path:\n"
          f"{path}\n")
    print(f"The contents of {path}:\n"
          f"{os.listdir(path)}")

[Data already unzipped]


To check if we are in the right directory

In [6]:
os.listdir(path)

['result_senses.txt',
 'semeval-2023-task-1-V-WSD-train-v1',
 'captions_train_clean_complete.csv',
 'README.md',
 'wordnet(individual tokens).csv']

Using `train_v1/` data

In [7]:
path_to_data = path + "semeval-2023-task-1-V-WSD-train-v1/"
data_labels = pd.read_csv(path_to_data + "train_v1/train.data.v1.txt", 
                sep = "\t", 
                names = ["target_word", "full_phrase"] + 
                ["image_%d"%index for index in range(10)])

In [8]:
data_labels.head()

,target_word,full_phrase,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg
3,bangalores,bangalores torpedo,image.58.jpg,image.59.jpg,image.64.jpg,image.57.jpg,image.55.jpg,image.56.jpg,image.62.jpg,image.63.jpg,image.61.jpg,image.60.jpg
4,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg


In [9]:
gold_labels = pd.read_csv(path_to_data + "train_v1/train.gold.v1.txt", 
                    names = ["gold_images"])

In [10]:
gold_labels.head()

,gold_images
0,image.0.jpg
1,image.20.jpg
2,image.35.jpg
3,image.55.jpg
4,image.75.jpg


Image Caption Data and WordNet senses

In [11]:
caption_data_labels = pd.read_csv(path + "captions_train_clean_complete.csv")
wordnet_senses_labels = pd.read_csv(path + "wordnet(individual tokens).csv")

In [12]:
caption_data_labels.head()

,File Name,Generated Caption
0,image.0.jpg,a black and white bird is standing in the air
1,image.1.jpg,a man with a red hat is standing in front of a...
2,image.10.jpg,a man is standing in a large white and white a...
3,image.100.jpg,a man and a woman are playing with a toy
4,image.1000.jpg,a person is riding a dirt path in a wooded area


In [13]:
wordnet_senses_labels.head()

,Complex_word,senses
0,moorhen swamphen,black gallinule that inhabits ponds and lakes ...
1,serinus genus,Old World finches; e.g. canaries and serins or...
2,pegmatite igneous,produced under conditions involving intense he...
3,bangalore torpedo,an industrial city in south central India (wes...
4,bonxie skua,gull-like jaeger of northern seas or gull-like...


## Creating the main `dataset`

In [14]:
dataset = data_labels.join(gold_labels)

In [15]:
dataset.head()

,target_word,full_phrase,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,gold_images
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg
3,bangalores,bangalores torpedo,image.58.jpg,image.59.jpg,image.64.jpg,image.57.jpg,image.55.jpg,image.56.jpg,image.62.jpg,image.63.jpg,image.61.jpg,image.60.jpg,image.55.jpg
4,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg


The only important dataframes are `caption_data_labels_renamed` and the newly merged `wordnet_senses_labels_renamed` into the main `dataset` dataframe

In [16]:
# renaming columns to match the `data_labels` dataframe and 
# to fit one standard format 
# (following the python naming convention - 
# https://peps.python.org/pep-0008/#naming-conventions)
#
caption_data_labels_renamed = caption_data_labels.rename(columns={
    "File Name": "images",
    "Generated Caption": "generated_caption"
    })
wordnet_senses_labels_renamed = wordnet_senses_labels.rename(columns={
    "Complex_word": "full_phrase"
})

In [17]:
caption_data_labels_renamed.head()

,images,generated_caption
0,image.0.jpg,a black and white bird is standing in the air
1,image.1.jpg,a man with a red hat is standing in front of a...
2,image.10.jpg,a man is standing in a large white and white a...
3,image.100.jpg,a man and a woman are playing with a toy
4,image.1000.jpg,a person is riding a dirt path in a wooded area


In [18]:
wordnet_senses_labels_renamed.head()

,full_phrase,senses
0,moorhen swamphen,black gallinule that inhabits ponds and lakes ...
1,serinus genus,Old World finches; e.g. canaries and serins or...
2,pegmatite igneous,produced under conditions involving intense he...
3,bangalore torpedo,an industrial city in south central India (wes...
4,bonxie skua,gull-like jaeger of northern seas or gull-like...


In [19]:
dataset = dataset.merge(wordnet_senses_labels_renamed, how="inner", on="full_phrase")

In [20]:
dataset.head()

,target_word,full_phrase,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,gold_images,senses
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg,black gallinule that inhabits ponds and lakes ...
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg,Old World finches; e.g. canaries and serins or...
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg,produced under conditions involving intense he...
3,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg,gull-like jaeger of northern seas or gull-like...
4,ixia,ixia genus,image.90.jpg,image.3.jpg,image.91.jpg,image.4.jpg,image.92.jpg,image.1.jpg,image.2.jpg,image.94.jpg,image.93.jpg,image.5.jpg,image.90.jpg,a monocotyledonous genus of the family Iridace...


Remove any missing data from the generated `senses`

In [21]:
for missing_data in np.where(dataset.isnull()):
    dataset = dataset.drop(missing_data)

In [22]:
# Check if the missing data are dropped (numpy array should appear empty)
np.where(dataset.isnull())

(array([], dtype=int64), array([], dtype=int64))

In [23]:
dataset.head()

,target_word,full_phrase,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,gold_images,senses
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg,black gallinule that inhabits ponds and lakes ...
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg,Old World finches; e.g. canaries and serins or...
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg,produced under conditions involving intense he...
3,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg,gull-like jaeger of northern seas or gull-like...
4,ixia,ixia genus,image.90.jpg,image.3.jpg,image.91.jpg,image.4.jpg,image.92.jpg,image.1.jpg,image.2.jpg,image.94.jpg,image.93.jpg,image.5.jpg,image.90.jpg,a monocotyledonous genus of the family Iridace...


# Word2Vec model

Reference (https://www.tensorflow.org/tutorials/text/word2vec)

In [24]:
import io
import re
import string
import tqdm

from tensorflow.keras import layers

In [25]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [26]:
# Define the vocabulary size and the number of words in a sequence.
VOCAB_SIZE = 4096
SEQUENCE_LENGTH = 10

In [27]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [28]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

In [29]:
# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

2023-03-17 00:01:43.480117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 00:01:43.526631: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 00:01:43.526797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 00:01:43.527306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [30]:
len(dataset) # Size of the dataset

11722

Reads the `full_phrase` and `senses` then combine them into `result_sense.txt`

In [31]:
import glob
# +------------------------------------------------------------+
# | had to do this hacky way to make jupyter run the for-loop  |
# | probably without repeating the first index of the `dataset`|
# +------------------------------------------------------------+
list_index = []
for index in range(len(dataset)):
    if index % 1 == 0:
        list_index.append(index)

# adds the final index that is not divisible by the mod number
if len(dataset) not in list_index:
    list_index.append(len(dataset)) 


if os.path.isfile(path + "result_senses.txt") == False:
    # i being the first index, y being the index after it
    print("Creating the senses text files...")
    for i, y in zip(tqdm.tqdm(list_index), list_index[1:]):
        file = open(path + "senses%d.txt"%y, "w")
        dataset_senses = ""
        for full_phrase, senses in zip(dataset.full_phrase[i:y], dataset.senses[i:y]):
            dataset_senses +=  full_phrase + " " + senses + " "
            file.write(dataset_senses)
        file.close()
        del full_phrase, senses  # clears the memory to avoid memory leak
    
    # Reference (https://stackoverflow.com/a/17749339)
    print("Combining senses text files into one...")
    senses_files = sorted(glob.glob(path + "senses*.txt"))
    with open(path + "result_senses.txt", "wb") as outfile:
        for f in tqdm.tqdm(senses_files):
            with open(f, "rb") as infile:
                outfile.write(infile.read())
            os.remove(f)
        
else:
    print("[result_senses.txt has already been created]")

[result_senses.txt has already been created]


In [32]:
text_ds = tf.data.TextLineDataset(path + "result_senses.txt").filter(lambda x: tf.cast(tf.strings.length(x), bool))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Vectorize sentences from the corpus

In [47]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

Call `TextVectorization.adapt` on the text dataset to create vocabulary.

In [48]:
vectorize_layer.adapt(text_ds.batch(1024))

Once the state of the layer has been adapted to represent the text corpus, the vocabulary can be accessed with `TextVectorization.get_vocabulary`. This function returns a list of all vocabulary tokens sorted (descending) by their frequency.

In [49]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'or', 'a', 'of', 'the', 'and', 'in', 'to', 'an', 'that', 'for', 'by', 'with', 'something', 'as', 'one', 'is', 'group', 'genus']


The `vectorize_layer` can now be used to generate vectors for each element in the `text_ds` (a `tf.data.Dataset`). Apply `Dataset.batch`, `Dataset.prefetch`, `Dataset.map`, and `Dataset.unbatch`.

In [51]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

Obtain sequences from the dataset

In [52]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

1


Inspect a few examples from `sequences`:

In [53]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[   1    1  332    1   10    1    1    6 3609    2] => ['[UNK]', '[UNK]', 'black', '[UNK]', 'that', '[UNK]', '[UNK]', 'and', 'lakes', 'or']


Generate training examples from sequences

In [39]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=VOCAB_SIZE,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 1/1 [00:00<00:00, 9137.92it/s]



targets.shape: (0,)
contexts.shape: (0,)
labels.shape: (0,)


## Training of Word2Vec

In [40]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
final_dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
final_dataset = final_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(final_dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.float64, name=None), TensorSpec(shape=(1024,), dtype=tf.float64, name=None)), TensorSpec(shape=(1024,), dtype=tf.float64, name=None))>


In [41]:
final_dataset = final_dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(final_dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.float64, name=None), TensorSpec(shape=(1024,), dtype=tf.float64, name=None)), TensorSpec(shape=(1024,), dtype=tf.float64, name=None))>


Subclassed word2vec model

In [42]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

Define loss function and compile model

In [43]:
embedding_dim = 128
word2vec = Word2Vec(VOCAB_SIZE, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

NameError: name 'num_ns' is not defined